In [117]:
import urllib
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

site = 'http://www.lada-vesta.net/'
folder_names = [
    ['0', 'Отзывы', 'f=25', 'LADA VESTA'],
    ['1', 'Двигатель', 'f=14', 'LADA VESTA'],
    ['2', 'Трансмиссия', 'f=15', 'LADA VESTA'],
    ['3', 'Ходовая', 'f=16', 'LADA VESTA'],
    ['4', 'Колеса', '', 'LADA VESTA'],
    ['5', 'Электрооборудование', 'f=17', 'LADA VESTA'],
    ['6', 'Мультимедиа', '', 'LADA VESTA'],
    ['7', 'Кузов', 'f=18', 'LADA VESTA'],
    ['8', 'Жидкости', 'f=19', 'LADA VESTA'],
    ['9', 'Сервис', 'f=42', 'LADA VESTA'],
    ['10', 'Эксплуатация', 'f=23', 'LADA VESTA'],
    ['11', 'Тюнинг', 'f=24', 'LADA VESTA']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [118]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [119]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}
        
        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем некоторые метаданные сообщений на текущей странице
            tables = soup.find_all('table')
            for next_table in tables:
                if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                    post_column = next_table.find('td')
                    post_date = post_column.get_text().strip()
                    post_data[re.sub('post', '', next_table['id'])] = [post_date]
            #Собираем сами сообщения
            divs = soup.find_all('div')
            for div in divs:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('postmenu_', div['id']):
                    author = div.find('a')
                    if author != None:
                        author_id = re.search('u=[0-9]+', author['href'])[0]
                        author_nickname = author.get_text()
                    else:
                        author_id = div.get_text().strip()
                        author_nickname = 'Guest'
                    post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        post_date = post_data[message_id][0]
                        author = post_data[message_id][1][1]
                    else:
                        post_date = ''
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                    if len(s) > 2:
                        theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
    return messages

In [120]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Time,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [121]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [122]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

The ipytelegram extension is already loaded. To reload it, use:
  %reload_ext ipytelegram


In [ ]:
%%telegram_send Next message
scraper(site, folder_names)

In [ ]:
import pandas as pd
mess = pd.read_csv('./LADA VESTA/LADA VESTA_10_Эксплуатация.csv', quotechar="'")
mess

In [8]:
folder_names = [['7', 'Кузов', 'f=18', 'LADA VESTA']]
folder = folder_names[0][2]
current_themes = collect_themes(site, folder)

In [ ]:
themes = []
flag = True

while flag:
    
    resp = urlopen(site_address) # скачиваем файл
    html = resp.read().decode('utf-8') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
    flag = False
    for link in soup.find_all('a'):
        if link.has_attr('href') and link.has_attr('id'):
            if re.search('thread_title_', str(link.get('id'))):
                s = link.get('href')
                s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                themes.append([link.get_text(), s])
        if not flag and link.has_attr('href') and link.has_attr('rel'):
            if link.get('rel')[0] == 'next':
                flag = True
                s = link.get('href')
                site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]

In [99]:
themes = [['Механика, робот или автомат?',
  'http://www.lada-vesta.net/showthread.php?t=47']]

In [ ]:
messages = []
for theme in tqdm(themes):
    theme_address = theme[1]
    thread = re.search('t=[0-9]+', theme_address)[0]
    flag = True
    theme_messages = []
    post_data = {}
    while flag:
        resp = urlopen(theme_address) # скачиваем файл
        html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
        #Собираем некоторые метаданные сообщений на текущей странице
        tables = soup.find_all('table')
        for next_table in tables:
            if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                post_column = next_table.find('td')
                post_date = post_column.get_text().strip()
                post_data[re.sub('post', '', next_table['id'])] = [post_date]
        #Собираем сами сообщения
        divs = soup.find_all('div')
        for div in divs:
            if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                continue
            if div.has_attr('id') and re.search('postmenu_', div['id']):
                author = div.find('a')
                if author != None:
                    author_id = re.search('u=[0-9]+', author['href'])[0]
                    author_nickname = author.get_text()
                else:
                    author_id = div.get_text().strip()
                    author_nickname = 'Guest'
                post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                print(re.sub('postmenu_', '', div['id']), post_data[re.sub('postmenu_', '', div['id'])])
            if div.has_attr('id') and re.search('post_message_', div['id']):
                message_id = re.sub('post_message_', '', div['id'])
                if message_id in post_data:
                    post_date = post_data[message_id][0]
                    author = post_data[message_id][1][1]
                else:
                    post_date = ''
                    author = ''
                for item in div.children:
                    if item.name == 'div':
                        item.clear()
                s = re.sub('[\']+', '`', div.get_text())
                s = '\'' + re.sub('[\s]+', ' ', s) + '\''
                if len(s) > 0:
                    theme_messages.append([post_date, author, s])
        # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
        # Если ссылка найдена, 
        flag = False 
        for link in soup.find_all('a'):
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
    messages.append(['\''+ theme[0] + '\'', theme_messages])

In [ ]:
messages

In [85]:
f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
with open(f_name, 'w', encoding='utf-8') as ouf:
    ouf.write('Code,Folder,Theme,Date,Author,Message\n')
    for message in messages:
        header = folder_name[:2] + [message[0]]
        for item in message[1]:
            ouf.write(','.join(header + item))
            ouf.write('\n')

In [69]:
folder_name = ['0', 'Отзывы', 'f=25', 'LADA VESTA']
f_name = folder_name[3] + '/' + folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv'
f_name

'LADA VESTA/LADA VESTA_0_Отзывы.csv'

In [ ]:
'''messages = []
theme_address = 'http://www.lada-vesta.net/showthread.php?t=9571'
resp = urlopen(theme_address) # скачиваем файл
html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
soup = BeautifulSoup(html, 'html.parser') # делаем суп
divs = soup.find_all('div')
for div in divs:
    if div.has_attr('id'):
        if re.search('post_message_', div['id']):
            for item in div.children:
                if item.name != 'div':
                    s = str(item).strip()
                    if len(s) > 0:
                        messages.append([s])
print(messages)'''